# Problem Statement: Study Assistant for Quiz Question Generation

In modern education, students often struggle to create effective quizzes for self-assessment. Manually generating relevant, diverse, and structured quiz questions can be time-consuming, leading to incomplete coverage of the material. This limitation hinders students' ability to effectively evaluate their understanding of complex topics.

Existing solutions, such as Retrieval-Augmented Generation (RAG) systems, often depend on external databases or vector storage mechanisms, which can be resource-intensive. While these approaches are powerful, they are unnecessary for localized study scenarios. As a result, there is a need for a lightweight, standalone solution that leverages modern language models to efficiently generate quiz questions without relying on external dependencies.

## Objective:

Develop a Study Assistant using LangChain that:

Summarizes study material into concise points.
Automatically generates multiple-choice quiz questions based on the summarized content.
Functions without the need for external retrieval mechanisms or vector databases.
The assistant should be user-friendly, adaptable to various educational topics, and designed to help students engage interactively with their learning material. This tool should be able to process study materials provided in course documents or any content the students choose to work with, and generate relevant quiz questions for review.

## Instructions for the Course Project:

### Using the Course Document:

Students can use the study material provided in the course document to generate quiz questions. The document will contain educational content on a specific topic, which should be summarized into concise points. Based on the summary, the system should automatically generate multiple-choice questions with four options, including the correct answer.

Loading Content into Prompts Using PyPDF2: PyPDF2 offers an efficient way to load study material from PDF documents. By using PyPDF2, you can extract the content from PDF files for further processing, such as summarizing the material and generating quiz questions.

Example: If the document is a PDF, you can use PyPDF2 to extract the content.

Example code to load content from a text file:

!pip install PyPDF2

import PyPDF2

#### Open the PDF file
with open("/content/Prompt Engineering.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)

#### Extract text from all pages
study_material = ""
for page in reader.pages:
    study_material += page.extract_text()

#### Now 'study_material' contains the text from the PDF
print(study_material)

In [17]:
# Importing the relevant libraries
import os
from dotenv import load_dotenv
import langchain
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import display, Markdown, Image
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from langchain_community.callbacks import get_openai_callback
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory, VectorStoreRetrieverMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableParallel, RunnableBranch
from langchain_core.runnables import chain
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory, SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter
import pandas as pd
from rich.console import Console
from rich.markdown import Markdown

In [18]:
# loading the environment variables
load_dotenv()

True

In [19]:
chatgpt = ChatOpenAI(model = "gpt-4o-mini", temperature=0)

In [50]:
import os
import PyPDF2

def extract_text_from_pdfs(directory):
    """
    Extracts text from all PDF files in the given directory.

    Args:
        directory: Path to the directory containing PDF files.

    Returns:
        A dictionary where keys are file names and values are extracted text.
    """

    text_from_pdfs = {}
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "rb") as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
            text_from_pdfs[filename] = text
    return text_from_pdfs

# Example usage:
directory = "./Langchain"
all_pdf_text = extract_text_from_pdfs(directory)

# Print the extracted text from each PDF (optional)
for filename, text in all_pdf_text.items():
    print(f"Text from {filename.upper()}:\n{text}\n")
    print("*"*50)

Text from PROMPT ENGINEERING.PDF:
What
is
Prompt
Engineering?
Prompt
engineering
is
a
practice
within
natural
language
processing
(NLP)
in
artificial
intelligence,
where
text
is
used
to
describe
the
task
the
AI
should
perform.
Guided
by
this
input,
the
AI
generates
an
output,
which
could
take
various
forms.
The
goal
is
to
use
human-understandable
text
to
interact
conversationally
with
models,
allowing
for
flexibility
in
the
model’ s
performance
due
to
the
task
description
embedded
in
the
prompt.
What
are
Prompts?
Prompts
are
detailed
descriptions
of
the
desired
output
from
an
AI
model.
They
represent
the
interaction
between
the
user
and
the
model
and
help
define
what
the
AI
is
expected
to
do.
The
effectiveness
of
prompt
engineering
largely
depends
on
how
well
the
prompt
is
designed
to
guide
the
model.
Examples
of
Prompt
Engineering
Prompts
in
large
language
models
(LLMs)
like
ChatGPT
or
GPT -3
can
range
from
simple
text
queries
to
complex
instructions.
The
key
to
effective
prompting
is

In [21]:
filename

'Prompt Engineering.pdf'

In [5]:
few_shot_examples = [
{"input":{
    "topic" :"Prompt Engineering for Agents",
    "study_material" :"""Prompt engineering involves designing and refining inputs to language models to achieve desired outputs.
                         In the context of agents, prompt engineering allows for better control over how an agent interacts with the environment and solves specific tasks.
                         This is particularly useful in domains like robotics and conversational AI.
                         By adjusting the structure and content of the prompts, users can enhance an agent's performance on specific tasks."""
},
"output":{
    "summary": """Prompt engineering refines inputs to language models for better output control.

                  In agent-based systems, it helps control agent behavior and task performance.

                  Useful in robotics and conversational AI.""",
    "Quiz_Question" : """What is the primary goal of prompt engineering in agent-based systems?

                        a) To optimize agent memory
                        b) To refine inputs for better output control
                        c) To improve agent hardware
                        d) To increase computational power
                        """,

    "Answer" : """b) To refine inputs for better output control""",
}
}]

In [44]:

few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("system","""Act as a study assistant for quiz generation.
                    Go through the topic and text in the input thoroughly
                    and generate output as shown in example_prompts.
                    Also generate as many multiple choice questions as possible which would help the students in preparation.
         
         Donot hallucinate or make up any question that's not a part of the content"""),
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)

print(few_shot_prompt.format())

System: Act as a study assistant for quiz generation.
                    Go through the topic and text in the input thoroughly
                    and generate output as shown in example_prompts.
                    Also generate as many multiple choice questions as possible which would help the students in preparation.
         
         Donot hallucinate or make up any question that's not a part of the content
Human: {'topic': 'Prompt Engineering for Agents', 'study_material': "Prompt engineering involves designing and refining inputs to language models to achieve desired outputs.\n                         In the context of agents, prompt engineering allows for better control over how an agent interacts with the environment and solves specific tasks.\n                         This is particularly useful in domains like robotics and conversational AI.\n                         By adjusting the structure and content of the prompts, users can enhance an agent's performance on specific 

In [45]:
inputs = [{"input":{"topic": filename.replace(".pdf",""),"study_material":text}} for filename, text in all_pdf_text.items()]


In [46]:
inputs

[{'input': {'topic': 'Prompt Engineering',
   'study_material': 'What\nis\nPrompt\nEngineering?\nPrompt\nengineering\nis\na\npractice\nwithin\nnatural\nlanguage\nprocessing\n(NLP)\nin\nartificial\nintelligence,\nwhere\ntext\nis\nused\nto\ndescribe\nthe\ntask\nthe\nAI\nshould\nperform.\nGuided\nby\nthis\ninput,\nthe\nAI\ngenerates\nan\noutput,\nwhich\ncould\ntake\nvarious\nforms.\nThe\ngoal\nis\nto\nuse\nhuman-understandable\ntext\nto\ninteract\nconversationally\nwith\nmodels,\nallowing\nfor\nflexibility\nin\nthe\nmodel’ s\nperformance\ndue\nto\nthe\ntask\ndescription\nembedded\nin\nthe\nprompt.\nWhat\nare\nPrompts?\nPrompts\nare\ndetailed\ndescriptions\nof\nthe\ndesired\noutput\nfrom\nan\nAI\nmodel.\nThey\nrepresent\nthe\ninteraction\nbetween\nthe\nuser\nand\nthe\nmodel\nand\nhelp\ndefine\nwhat\nthe\nAI\nis\nexpected\nto\ndo.\nThe\neffectiveness\nof\nprompt\nengineering\nlargely\ndepends\non\nhow\nwell\nthe\nprompt\nis\ndesigned\nto\nguide\nthe\nmodel.\nExamples\nof\nPrompt\nEngineerin

In [47]:
chain = (
    few_shot_prompt
          |
        chatgpt
          |
    StrOutputParser()
)

In [48]:
response = chain.map().invoke(inputs)

In [49]:
display(Markdown(response[0]))

Summary:                                                                                                           
Prompt engineering involves designing and refining inputs to language models to achieve desired outputs. In the    
context of agents, it allows for better control over how an agent interacts with the environment and solves        
specific tasks. This is particularly useful in domains like robotics and conversational AI. By adjusting the       
structure and content of the prompts, users can enhance an agent's performance on specific tasks.                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Quiz Questions:                                                                                                    

 1 What is prompt engineering primarily concerned with?                                                            
   a) Designing hardware for agents                                                                                
   b) Refining inputs to language models                                                                           
   c) Increasing the speed of language models                                                                      
   d) Developing new programming languages                                                                         
   Answer: b) Refining inputs to language models                                                                   
 2 In which domains is prompt engineering particularly useful?                                                     
   a) Web development and data analysis                                                                            
   b) Robotics and conversational AI                                                                               
   c) Graphic design and video editing                                                                             
   d) Network security and database management                                                                     
   Answer: b) Robotics and conversational AI                                                                       
 3 How does prompt engineering affect an agent's performance?                                                      
   a) It has no effect on performance                                                                              
   b) It enhances performance on specific tasks                                                                    
   c) It complicates the agent's decision-making process                                                           
   d) It reduces the agent's ability to learn                                                                      
   Answer: b) It enhances performance on specific tasks                                                            
 4 What is one of the main benefits of adjusting the structure and content of prompts?                             
   a) It makes the agent more complex                                                                              
   b) It allows for better control over agent interactions                                                         
   c) It decreases the agent's efficiency                                                                          
   d) It eliminates the need for training data                                                                     
   Answer: b) It allows for better control over agent interactions                                                 
 5 What does prompt engineering allow users to do in the context of agents?                                        
   a) Increase the agent's physical capabilities                                                                   
   b) Control how the agent interacts with the environment                                                         
   c) Change the agent's programming language         